[![colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davemlz/eemont/blob/master/docs/tutorials/004-Computing-Spectral-Indices-Landsat-8.ipynb)
[![Open in SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/davemlz/eemont/blob/master/docs/tutorials/004-Computing-Spectral-Indices-Landsat-8.ipynb)
[![Open in Planetary Computer](https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft)](https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/davemlz/eemont&urlpath=lab/tree/eemont/docs/tutorials/004-Computing-Spectral-Indices-Landsat-8.ipynb&branch=master)

# Computing Spectral Indices in Landsat 8
_Tutorial created by **David Montero Loaiza**_: [GitHub](https://github.com/davemlz) | [Twitter](https://twitter.com/dmlmont)

- GitHub Repo: [https://github.com/davemlz/eemont](https://github.com/davemlz/eemont)
- PyPI link: [https://pypi.org/project/eemont/](https://pypi.org/project/eemont/)
- Conda-forge: [https://anaconda.org/conda-forge/eemont](https://anaconda.org/conda-forge/eemont)
- Documentation: [https://eemont.readthedocs.io/](https://eemont.readthedocs.io/)
- More tutorials: [https://github.com/davemlz/eemont/tree/master/docs/tutorials](https://github.com/davemlz/eemont/tree/master/docs/tutorials)

## Let's start!

If required, please uncomment:

In [25]:
#!pip install eemont
#!pip install geemap

Import the required packages.

In [26]:
import ee, eemont, datetime, geemap

Authenticate and Initialize Earth Engine and geemap.

In [27]:
Map = geemap.Map()

Point of interest.

In [28]:
point = ee.Geometry.Point([-74.0592,11.3172])

Get, filter, mask clouds and scale the image collection.

In [29]:
L8 = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
      .filterBounds(point)
      .sort('CLOUD_COVER')
      .first()
      .maskClouds()
      # Incorporate logic to handle potential STAC metadata errors, this will prevent potential errors.
      .scaleAndOffset(collection_id='LANDSAT/LC08/C02/T1_L2') # Provide specific collection ID
     )

EEException: ImageCollection.load: ImageCollection asset 'LANDSAT/LC08/C01/T1_SR' not found (does not exist or caller does not have access).

## Spectral Indices

Spectral indices can be computed for Sentinel-2 and Landsat Products using the `spectralIndices()` method.

In [ ]:
L8 = L8.spectralIndices()

By default, the NDVI is computed and added as a new band.

In [ ]:
L8.bandNames().getInfo()

If required, another index can be computed:

In [ ]:
L8 = L8.spectralIndices('BAI')

This new index is now added as another new band.

In [ ]:
L8.bandNames().getInfo()

Multiple indices can also be computed.

In [ ]:
L8 = L8.spectralIndices(['EVI','GNDVI','RVI'])

These indices are now added as new bands:

In [ ]:
L8.bandNames().getInfo()

Group of indices can also be added by the group name. The available options are: vegetation, water, burn, snow or all (compute all spectral indices):

In [ ]:
L8 = L8.spectralIndices('water') # Computes NDWI and MNDWI

The computed indices are now added as new bands:

In [ ]:
L8.bandNames().getInfo()

## Visualization

Now, let's plot two of the computed indices: GNDVI and NDWI.

Let's define the visualization parameters for the GNDVI:

In [ ]:
visParamsGNDVI = {
  'min': 0.0,
  'max': 1.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}

Let's define the visualization parameters for the NDWI:

In [ ]:
visParamsNDWI = {
  'min': 0.0,
  'max': 1.0,
  'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff'],
}

Use `geemap` to display results:

In [ ]:
Map.centerObject(point,9)
Map.addLayer(L8.select('GNDVI'),visParamsGNDVI,'GNDVI')
Map.addLayer(L8.select('NDWI'),visParamsNDWI,'NDWI')
Map.add_colorbar(visParamsGNDVI['palette'], labels = 'GNDVI')
Map.add_colorbar(visParamsNDWI['palette'], labels = 'NDWI')
Map